## Transfer Learning RESENT50 using Keras

Please download the dataset from the below url

In [33]:
import tensorflow as tf 

if tf.test.gpu_device_name(): 
    

    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))

else:

    print("Please install GPU version of TF")

Default GPU Device:/device:GPU:0


In [34]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [35]:
tf.__version__

'2.3.0'

In [42]:
# import the libraries as shown below

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob


In [43]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = r'C:\Users\50021726\Desktop\Ineuron\catdogdata\training_set'
valid_path = r'C:\Users\50021726\Desktop\Ineuron\catdogdata\test_set'


In [44]:
# Import the Vgg 16 library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights

ResNet50 = ResNet50(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)




94773248/94765736 [==============================] - 197s 2us/step


In [45]:
# don't train existing weights
for layer in ResNet50.layers:
    layer.trainable = False

In [46]:
  # useful for getting number of output classes
folders = glob(r'C:\Users\50021726\Desktop\Ineuron\catdogdata\training_set/*')
folders

['C:\\Users\\50021726\\Desktop\\Ineuron\\catdogdata\\training_set\\cats',
 'C:\\Users\\50021726\\Desktop\\Ineuron\\catdogdata\\training_set\\dogs']

In [48]:
# our layers - you can add more if you want
x = Flatten()(ResNet50.output)
x

<tf.Tensor 'flatten_2/Reshape:0' shape=(None, 100352) dtype=float32>

In [49]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=ResNet50.input, outputs=prediction)

In [50]:

# view the structure of the model
model.summary()


Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
_______________________________________________________________________________________

In [51]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


In [52]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [53]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory(r'C:\Users\50021726\Desktop\Ineuron\catdogdata\training_set',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 8005 images belonging to 2 classes.


In [54]:
test_set = test_datagen.flow_from_directory(r'C:\Users\50021726\Desktop\Ineuron\catdogdata\test_set',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                          class_mode = 'categorical')

Found 2023 images belonging to 2 classes.


In [16]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [55]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Epoch 1/10
251/251 [==============================] - 115s 458ms/step - loss: 0.9191 - accuracy: 0.5935 - val_loss: 0.9088 - val_accuracy: 0.5759
Epoch 2/10
251/251 [==============================] - 101s 402ms/step - loss: 0.7726 - accuracy: 0.6202 - val_loss: 0.7931 - val_accuracy: 0.5858
Epoch 3/10
251/251 [==============================] - 101s 402ms/step - loss: 0.7931 - accuracy: 0.6319 - val_loss: 0.5906 - val_accuracy: 0.7138
Epoch 4/10
251/251 [==============================] - 110s 437ms/step - loss: 0.7378 - accuracy: 0.6551 - val_loss: 0.5844 - val_accuracy: 0.7291
Epoch 5/10
251/251 [==============================] - 117s 468ms/step - loss: 0.7926 - accuracy: 0.6399 - val_loss: 0.5990 - val_accuracy: 0.7301
Epoch 6/10
251/251 [==============================] - 116s 464ms/step - loss: 0.6467 - accuracy: 0.6913 - val_loss: 1.2065 - val_accuracy: 0.5363
Epoch 7/10
251/251 [==============================] - 117s 466ms/step - loss: 0.7646 - accuracy: 0.6785 - val_loss: 0.5788 -

In [18]:
# plot the loss
%matplotlib
import matplotlib.pyplot as plt
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

Using matplotlib backend: TkAgg


In [56]:
# save it as a h5 file


from tensorflow.keras.models import load_model

model.save('resnet50.h5')

In [57]:

y_pred = model.predict(test_set)


In [58]:
y_pred

array([[4.7092208e-01, 5.2907789e-01],
       [7.0319261e-04, 9.9929678e-01],
       [1.1319838e-01, 8.8680166e-01],
       ...,
       [5.4438356e-02, 9.4556165e-01],
       [1.4800390e-02, 9.8519957e-01],
       [3.3216190e-01, 6.6783816e-01]], dtype=float32)

In [59]:
import numpy as np
y_pred = np.argmax(y_pred, axis=1)

In [60]:
y_pred

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [61]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [63]:
model=load_model('resnet50.h5')

In [64]:
img=image.load_img(r'C:\Users\50021726\Desktop\Ineuron\catdogdata\test_set\dogs\dog.4011.jpg',target_size=(224,224))



In [65]:
x=image.img_to_array(img)
x

array([[[ 54.,  69.,  28.],
        [108., 126.,  86.],
        [ 90., 111.,  72.],
        ...,
        [ 25.,  50.,  29.],
        [ 29.,  56.,  41.],
        [ 31.,  57.,  44.]],

       [[ 44.,  61.,  19.],
        [ 45.,  61.,  22.],
        [ 39.,  55.,  16.],
        ...,
        [ 21.,  48.,  31.],
        [ 24.,  55.,  37.],
        [ 21.,  52.,  34.]],

       [[ 50.,  68.,  26.],
        [ 53.,  68.,  29.],
        [ 52.,  62.,  25.],
        ...,
        [ 26.,  53.,  38.],
        [ 23.,  55.,  34.],
        [ 43.,  76.,  55.]],

       ...,

       [[245., 246., 241.],
        [246., 247., 242.],
        [247., 248., 243.],
        ...,
        [194., 193., 199.],
        [191., 191., 193.],
        [199., 199., 199.]],

       [[247., 248., 243.],
        [248., 249., 244.],
        [248., 249., 244.],
        ...,
        [194., 193., 199.],
        [190., 190., 192.],
        [197., 197., 197.]],

       [[250., 251., 246.],
        [250., 251., 246.],
        [249., 2

In [66]:
x.shape

(224, 224, 3)

In [67]:
x=x/255

In [68]:
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
img_data.shape

(1, 224, 224, 3)

In [ ]:
model.predict(img_data)

In [30]:
a=np.argmax(model.predict(img_data), axis=1)

In [31]:
a

array([0], dtype=int64)

In [32]:
if(a==0):
    print("The image classified is dog")
else:
    print("The image classified is cat")

The image classified is dog
